# Estimate car vs bus travel time

* Pull out parallel routes.
* Make car travel down same route as the bus.
* `osmx` snaps to nodes, but even for every 5th bus stop, it's snapping to same node.
* `osrm` wasn't able to be installed in Hub
* `valhalla`? Kuan Butt's blog?

#### Quick and Dirty Approach
* Pull parallel routes
* New query that grabs stop sequences for each trip
* **Pare down** trips, keep only 1 trip for the route (pick longest one), ignore short trips
* Do above step outside of query; query returning distinct is keeping it across trips and mixing up stop sequences, and weird results come out
* Add `service_hours` for that trip, in `view.gtfs_fact_daily_trips`, this is GTFS scheduled 
* At least be able to pull that same trip, or if it's a different day, pull it for the same time?
* Based on distance traveled, estimate car travel time with some assumptions (35, 40 mph?)
* For now, estimate car travel with lower mph assumption, so that some viable routes can be pulled. Don't want bus to look worse than it is (mid-day, free-flowing), and compare it to car travel (which is probably estimated during free-flowing too)

Later, swap out car travel time estimation with other approaches. Maybe use Google API to do requests.

In [1]:
#https://stackoverflow.com/questions/55162077/how-to-get-the-driving-distance-between-two-geographical-coordinates-using-pytho
import geopandas as gpd
import os
import pandas as pd

os.environ["CALITP_BQ_MAX_BYTES"] = str(130_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
from siuba import *

import shared_utils
import utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0317 18:51:24.661380666    1000 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0317 18:51:29.853199843    1000 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [ ]:
'''
SELECTED_DATE = "2022-2-8"

tbl_stop_times = (
    tbl.views.gtfs_schedule_dim_stop_times()
    >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
              _.calitp_deleted_at > SELECTED_DATE, 
             )
)


daily_stop_times = (
    tbl.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.service_date == SELECTED_DATE, 
          _.is_in_service == True)
    >> filter(_.calitp_itp_id==182)
    >> left_join(_, tbl_stop_times,
              # also added url number to the join keys ----
             ["calitp_itp_id", "calitp_url_number", "trip_id"])
    >> select(_.calitp_itp_id,
           _.trip_id, _.route_id, _.stop_id, _.stop_sequence, 
           _.service_hours, _.trip_first_departure_ts, _.trip_last_arrival_ts
          )    
    >> inner_join(_, 
                  (tbl.views.gtfs_schedule_dim_stops()
                   >> select(_.calitp_itp_id,
                            _.stop_id, _.stop_lon, _.stop_lat,
                            )
                  ), on = ["calitp_itp_id", "stop_id"]
    )
    >> distinct()
    >> collect()
)
'''

In [ ]:
#daily_stop_times.to_parquet("./data/metro_routes.parquet")

In [2]:
routes_with_stops = pd.read_parquet("./data/metro_routes.parquet")

In [3]:
gdf = shared_utils.utils.download_geoparquet(utils.GCS_FILE_PATH, 
                                             "parallel_or_intersecting")

gdf = gdf[gdf.parallel==1].reset_index(drop=True)

# Start with LA Metro
gdf = gdf[gdf.itp_id==182].reset_index(drop=True)

In [4]:
def select_parallel_routes(df, parallel_info):
    df = df.rename(columns = {"calitp_itp_id": "itp_id"})
    
    gdf = (df[df.route_id.isin(parallel_info.route_id)]
            .sort_values(["itp_id", "route_id", "stop_sequence"])
            .drop_duplicates(subset=["itp_id", "route_id", "stop_sequence"])
            .reset_index(drop=True)
           )
    
    gdf = shared_utils.geography_utils.create_point_geometry(
        gdf, longitude_col = "stop_lon", latitude_col = "stop_lat",
    )
    
    return gdf

parallel = select_parallel_routes(routes_with_stops, gdf)

In [5]:
#https://stackoverflow.com/questions/25055712/pandas-every-nth-row
# Maybe not use every bus stop, since bus stops are spaced fairly closely
# Maybe every other, every 3rd? want to mimic the bus route, do not want
# to stray too far
#df = df.iloc[::3]

Don't like how `osmx` is returning the same nodes for bus stops, even at every 5th bus stop.

`osrm` doesn't install bc of some `GDAL` dependencies.

Can Google API be used? But need to check terms and conditions if we can make requests to calculate travel time or even grab speed limits through the
[Python package](https://github.com/googlemaps/google-maps-services-python)

At minimum, can calculate distance between stops, sum it up, and for cars, set an assumption of 30 mph or 45 mph. If we can't use Google API to grab speed limit, then we will hard code it.

In [6]:
def calculate_distance_traveled(df):
    group_cols = ["itp_id", "route_id"]
    sort_cols = group_cols + ["stop_sequence"]
    
    df = df.to_crs(shared_utils.geography_utils.CA_StatePlane)
    
    # Distance traveled
    df = df.assign(
        # Previous geometry
        start = (df.sort_values(sort_cols)
                 .groupby(group_cols)["geometry"]
                 .apply(lambda x: x.shift(1))),
        end = (df.sort_values(sort_cols)
               .groupby(group_cols)["geometry"]
               .apply(lambda x: x.shift(0))
              )
    )
    
    df = df.assign(
        feet_traveled = df.end.distance(df.start) 
    ).drop(columns = ["start", "end"])
        
    return df
            

In [7]:
df = calculate_distance_traveled(parallel)

In [8]:
def calculate_time_traveled(df):
    # Use a set of assumptions
    
    AVG_SPEED = 40
    
    df = df.assign(
        max_stop = (df.groupby(["itp_id", "route_id", "trip_id"])
                    ["stop_sequence"].transform("max"))
    )
    
    df2 = shared_utils.geography_utils.aggregate_by_geography(
        df,
        group_cols = ["itp_id", "route_id", "trip_id", 
                     "trip_first_departure_ts", "trip_last_arrival_ts"],
        sum_cols = ["feet_traveled"], 
        mean_cols = ["service_hours", "max_stop"]
    )
    
    df2 = df2.assign(
        miles_traveled = df2.feet_traveled.divide(
            shared_utils.geography_utils.FEET_PER_MI)
    
    )
    
    # speed = distance / time
    # time = distance / speed
    df2 = df2.assign(
        car_trip_time_hr = df2.miles_traveled.divide(AVG_SPEED),
        departure_hr = pd.to_datetime(df2.trip_first_departure_ts, unit='s').dt.hour                                        
    ).drop(columns = "feet_traveled")
        
    return df2

In [9]:
df2 = calculate_time_traveled(df)

In [10]:
t1 = df.copy()

In [11]:
import numpy as np
#https://stackoverflow.com/questions/17578115/pass-percentiles-to-pandas-agg-function
def q20(x):
    return x.quantile(0.2)

def q25(x):
    return x.quantile(0.25)


#https://stackoverflow.com/questions/44023770/pandas-getting-rid-of-the-multiindex
t2 = t1.groupby(['itp_id', 'route_id']).agg({'service_hours': [q20, q25]})
t2.columns = t2.columns.map(lambda x: x[1]) 
t2 = t2.reset_index()
t2

,itp_id,route_id,q20,q25
0,182,10-13153,1.183333,1.200000
1,182,106-13153,1.166667,1.241667
2,182,117-13153,1.233333,1.233333
3,182,120-13153,1.950000,1.950000
4,182,125-13153,1.550000,1.550000
...,...,...,...,...
73,182,901-13153,0.883333,0.883333
74,182,910-13153,0.833333,0.904167
75,182,92-13153,1.783333,2.066667
76,182,94-13153,1.450000,1.450000


Which trip should be selected?

It does appear that `max_stop` differs even for the same route. Not so clear what short vs long trips are. Should a trip with of average `service_hours` be selected? or average `miles_traveled` to represent a typical trip? 

But, typical trip is probably combination of mid-day service with one that ran near the average of `service_hours` or `miles_traveled`? Don't want to pull short trips because those may take place during peak.

In [ ]:
def select_one_trip(df):
    # Not sure why across trip_ids, 
    # for the same route_id, there are differing max_stop_sequence
    # Use longest route (max stop sequence)?
    # Use median or mean service hours or miles traveled?
    group_cols = ["itp_id", "route_id"]
    
    # Should there be a check that there are mid-day trips for that route_id?
    # Select trip by departure_hr
    hour_order = [
        12, 11, 13, 10, 14, # ideally we want mid-day
        15, 7, 20, 6, 21, # but, can move into earlier PM or early AM
        0, 1, 2, 3, 4, 5, 22, 23, # owl service
        8, 9, # AM peak 
        16, 17, 18, 19, # PM peak
    ]
    for i in range(0, 24):
        if i == 0:
            df['selection_rank'] = df.apply(
                lambda x: hour_order[i] if x.departure_hr == i 
                else 0, axis=1) 
        else:
            df['selection_rank'] = df.apply(
                lambda x: hour_order[i] if x.departure_hr == i 
                else x.selection_rank, axis=1) 
    
    # Select a trip that is somewhere in 75th-80th percentile
    df['p25'] = df.groupby(["itp_id", "route_id"])["service_hours"].quantile(0.25) 
    df['p20'] = df.groupby(["itp_id", "route_id"])["service_hours"].quantile(0.2)
    
    df = df.assign(
        p25 = df.groupby(["itp_id", "route_id"])["p25"].transform("max"), 
        p20 = df.groupby(["itp_id", "route_id"])["p20"].transform("max"), 
    )
    
    df['faster_trip'] = df.apply(lambda x: 
                                 1 if ((x.service_hours <= x.p25) and 
                                       (x.service_hours >= x.p20))
                                 else 0, axis=1)
    
    # Now select the first trip
    df2 = (df.sort_values(["itp_id", "route_id", "selection_rank"])
           .drop_duplicates(subset=["itp_id", "route_id"])
           .drop(columns = ["selection_rank", "max_stop"])
          )
    
    return df2

In [ ]:
df3 = select_one_trip(df2)

Comparison should be against bus's travel time along that route.

Can we pick one that is midday, one of the faster trips? Should be probably around 75th or 80th percentile.

Then see how long it takes for the bus to make that trip.

Actually, that travel time is in the data warehouse. Do another query, grab all the travel times, see if one can be selected for 75th or 80th percentile and if it's still less than 2x car trip time, then it can be selected as "viable parallel" route.

`views.gtfs_schedule_fact_daily_trips` has the `service_hours` column...should grab that in original query because later I drop a bunch of trips to get down to unique route, and select longest trip.